In [1]:
# 선언부
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
date = '2017.03.28'
url_part1 = '''http://dart.fss.or.kr/dsac001/mainAll.do?selectDate='''
url_part2 = '''&sort=&series=&mdayCnt=0&currentPage='''

url = url_part1 + date + url_part2

res = requests.get(url)
soup = bs(res.text, 'html.parser')

totNum = soup.find_all('p')[8].text
totNum = re.sub("\\[.*\\] \\[","",totNum)
totNum = re.sub(r"총","",totNum)
totNum = re.sub(r"건]","",totNum)
totNum = int(totNum.strip())

if totNum % 100 == 0 :
    totPage = totNum//100
else : 
    totPage = totNum // 100 +1

company_DF = pd.DataFrame([])
tmpDF = pd.DataFrame([])
tmpDF = tmpDF.append({'pubTime':'','coName':'','coCat':'','coID':'','content':'','pubDate':'','rcpNo':''},ignore_index=True)

for k in range(1,totPage+1) : 
    req = requests.get(url+str(k))
    soupData = bs(req.text, 'html.parser')

    data = soupData.find_all('tr')
    getCurLen = len(soupData.find_all('tr')) # 최댓값(목차 포함이므로 2번째줄부터 ㅆㄴ다)
    for i in range(1,getCurLen) :
        tmpCompany = data[i].find_all('td')
        tmpDF.at[0,'pubTime'] = tmpCompany[0].text.strip()
        tmpDF.at[0,'coName'] = tmpCompany[1].text.strip()
        tmpDF.at[0,'coCat'] = tmpCompany[1].find('img').get('title')
        tmpDF.at[0,'coID'] = tmpCompany[1].find('a').get('href')[-8:]
        tmpDF.at[0,'content'] = re.sub(r"[\t\r\n]","",tmpCompany[2].text.strip())
        tmpDF.at[0,'pubDate'] = tmpCompany[4].text.strip()
        tmpDF.at[0,'rcpNo'] = tmpCompany[2].find('a').get('id')[2:]

        company_DF = company_DF.append(tmpDF, ignore_index = True)

        for j in range(len(tmpDF.columns)) :
            tmpDF.iat[0,j] = ''
    
    

In [3]:
company_DF

,coCat,coID,coName,content,pubDate,pubTime,rcpNo
0,코넥스시장,01109690,포레스팅블록체인,감사보고서제출,2017.03.28,18:58,20170328601049
1,코스닥시장,00364403,쏠리드,불성실공시법인지정예고 (공시불이행),2017.03.28,18:46,20170328901048
2,코스닥시장,00308559,코디,주권매매거래정지 (불성실공시법인 지정),2017.03.28,18:35,20170328901037
3,코스닥시장,00308559,코디,불성실공시법인지정 (공시불이행),2017.03.28,18:35,20170328901047
4,기타법인,00207223,위너지스,주권매매거래정지 (불성실공시법인 지정),2017.03.28,18:29,20170328901039
...,...,...,...,...,...,...,...
889,기타법인,00178806,바이더웨이,임원의변동,2017.03.28,07:00,20170327000959
890,기타법인,01083000,케이엔에프인터내셔날,[기재정정]감사보고서 (2016.12),2017.03.28,07:00,20170327000967
891,기타법인,01078868,케이피엑스글로벌,감사보고서 (2016.12),2017.03.28,07:00,20170327000958
892,기타법인,00186513,코리아세븐,동일인등출자계열회사와의상품ㆍ용역거래,2017.03.28,07:00,20170327000962
